In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
from arctic import Arctic, CHUNK_STORE
lib = Arctic('localhost')
fund = lib.get_library('fund')

/home/azureuser/miniconda3/envs/fundopt/lib/python3.9/site-packages/arctic-1.80.0-py3.9.egg/arctic/store/_pandas_ndarray_store.py:8: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version


In [3]:
import pandas as pd

In [4]:
symbols = fund.list_symbols()

In [67]:
def get_fund_split_data(symbol):
    url = f'http://fundf10.eastmoney.com/fhsp_{symbol}.html'
    res = requests.get(url)
    html = BeautifulSoup(res.text)
    split_data = html.find('table', attrs={'class':'w782 comm fhxq'})
    split = pd.read_html(str(split_data))[0]
    
    if split.iloc[0,0] == '暂无拆分信息!':
        return pd.DataFrame()
    
    rename = {'年份' : 'year', 
              '拆分折算日' : 'date', 
              '拆分类型' : 'splitType', 
              '拆分折算比例' : 'amount'}

    split = split.rename(columns=rename)
    
    split['date'] = pd.to_datetime(split['date'])
    split['amount'] = split['amount'].str.split(':', expand=True)[1].astype(float)

    split = split.drop(['year', 'splitType'], axis=1).set_index('date').sort_index()
    
    return split

In [73]:
def get_fund_div_data(symbol):
    
    url = 'https://stock.finance.sina.com.cn/fundInfo/api/openapi.php/FundPageInfoService.tabfh'

    data_input = {
        'symbol' : symbol, 
        'format' : 'json',
    }

    resp = requests.get(url, params=data_input)
    data = resp.json()
    fhdata = data['result']['data']['fhdata']
    
    if len(fhdata)==0:
        return pd.DataFrame()
    
    div = pd.DataFrame(fhdata).astype({'mffh':float})
    
    div = div[div['mffh'] > 0.0]
    
    div = div.drop('fhr', axis=1)

    RENAME = {
        'djr' : 'date',
        'mffh' : 'amount',
    }

    div = div.rename(columns = RENAME)

    div['date'] = pd.to_datetime(div['date'])
   
    # sort div data frame from old to new
    div = div.set_index('date').sort_index()
    
    return div

In [74]:
def get_fund_adj_data(symbol):
    split = get_fund_split_data(symbol).assign(type='split')
    div = get_fund_div_data(symbol).assign(type='div')
    adj = pd.concat([split, div]).sort_index()

    return adj

In [6]:
import tqdm

In [ ]:
for symbol in tqdm.tqdm(symbols[2131:]):
    adj = get_fund_adj_data(symbol)
    adj.to_csv(f'adj_temp/{symbol}.csv')

 11%|█         | 183/1668 [04:42<34:59,  1.41s/it]

In [89]:
symbol

'003681'

In [90]:
get_fund_adj_data(symbol)

,type,amount
2018-12-19,div,0.037
2020-01-02,div,0.052
2020-10-26,div,0.040
2020-11-18,div,0.030


In [91]:
symbols.index(symbol)

2131